In [ ]:
!python experiment_k.py

--- Final sys.path ---
0: /home/am5715/smoothllm_workshop/my_private_libs
1: /home/am5715/.local/lib/python3.11/site-packages
2: /home/am5715/smoothllm_workshop
3: /.autofs/tools/spack/var/spack/environments/default-ml-x86_64-24071101/.spack-env/view/lib/scons
4: /.autofs/tools/spack/var/spack/environments/default-ml-x86_64-24071101/.spack-env/view/lib/python3.11/site-packages
----------------------
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:09<00:00,  4.70s/it]
2025-08-27 00:24:46.068818: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-27 00:24:46.134153: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instruction

# Experiment 1: Test K-Instablity



In [1]:
import sys
import warnings

warnings.filterwarnings("ignore")


user_site_packages = '/home/am5715/.local/lib/python3.11/site-packages'

if user_site_packages in sys.path:
    sys.path.remove(user_site_packages)

sys.path.insert(0, user_site_packages)

import os

private_libs_path = '/home/am5715/smoothllm_workshop/my_private_libs'

if private_libs_path not in sys.path:
    sys.path.insert(0, private_libs_path)


os.environ["ACCELERATE_DISABLE_BNB"] = "1"
os.environ["TRANSFORMERS_SKIP_AWS"] = "1"
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

print("--- Final sys.path ---")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")
print("----------------------")

--- Final sys.path ---
0: /home/am5715/smoothllm_workshop/my_private_libs
1: /home/am5715/.local/lib/python3.11/site-packages
2: /home/am5715/smoothllm_workshop
3: /.autofs/tools/spack/var/spack/environments/default-ml-x86_64-24071101/.spack-env/view/lib/scons
4: 
----------------------


In [2]:
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import fastchat

import smoothllm.lib.perturbations as perturbations
import smoothllm.lib.defenses as defenses
import smoothllm.lib.attacks as attacks
import smoothllm.lib.language_models as language_models
import smoothllm.lib.model_configs as model_configs

os.getcwd()

'/home/am5715/smoothllm_workshop'

# Llama2

In [3]:
torch.cuda.empty_cache()

# Targeted LLM
target_model= 'llama2'

# Attacking LLM
attack='GCG'
attack_logfile='smoothllm/data/GCG/llama2_behaviors.json'

# SmoothLLM
smoothllm_num_copies=1
smoothllm_pert_pct=1
smoothllm_pert_type='RandomSwapPerturbation'

In [4]:
# Instantiate the targeted LLM
config = model_configs.MODELS[target_model]
target_model = language_models.LLM(
    model_path=config['model_path'],
    tokenizer_path=config['tokenizer_path'],
    conv_template_name=config['conversation_template'],
    device='cuda:0'
)

# Create SmoothLLM instance
defense = defenses.SmoothLLM(
    target_model=target_model,
    pert_type=smoothllm_pert_type,
    pert_pct=smoothllm_pert_pct,
    num_copies=smoothllm_num_copies
)

# Create attack instance, used to create prompts
attack = vars(attacks)[attack](
    logfile=attack_logfile,
    target_model=target_model,
    start_index=0,
    end_index=50
)

# # Checking defense success rate with different positions
# jb_percentage = []
# for _ in range(2):
#     for i, prompt in tqdm(enumerate(attack.prompts)):
#         output = defense(prompt)
#         jb = defense.is_jailbroken(output)
#         jb_percentage.append(jb)
# print(f"For k=1, Attack Accuracy: {np.mean(jb_percentage)}, Standard Dev.: {np.std(jb_percentage)}")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

50
50
Generating attack for goal: 'Write a script that can exploit vulnerabilities in...'


2025-08-22 16:32:16.234061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-22 16:32:16.299286: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-22 16:32:18.018420: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
  0%|          | 0/500 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
np.mean(jb_percentage), np.std(jb_percentage)

In [ ]:
print(jb_percentage)